In [1]:
import rasterio
import numpy as np 
import pandas as pd
import geopandas as gpd
import pyproj
from rasterstats import zonal_stats
from rasterio.mask import mask
from rasterio import features
import warnings
warnings.filterwarnings("ignore")
print("BE CAREFULL warning are masked !!")

BE CAREFULL warning are masked !!


import dataset

In [2]:
EnMAP_glob_path = 'images/EnMAP_resampled_notglob.tif'

#glob_mask_path = 'images/4.tif'

shp_path = 'images/DECL_23_crop.shp'

In [3]:
with rasterio.open(EnMAP_glob_path, 'r') as src:
    print(src.profile)

{'driver': 'GTiff', 'dtype': 'float32', 'nodata': None, 'width': 1276, 'height': 1384, 'count': 224, 'crs': CRS.from_wkt('PROJCS["WGS 84 / UTM zone 31N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",3],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","32631"]]'), 'transform': Affine(30.0, 0.0, 570945.0,
       0.0, -30.0, 5627835.0), 'blockxsize': 1276, 'blockysize': 1, 'tiled': False, 'compress': 'lzw', 'interleave': 'pixel'}


Check crs

In [4]:
gdf = gpd.read_file(shp_path)
src = rasterio.open(EnMAP_glob_path, 'r')

crs_vector = pyproj.CRS(gdf.crs)
crs_raster = pyproj.CRS(src.crs)

if crs_vector == crs_raster:
    print(f'CRS are the same : EPSG:{crs_vector} = EPSG:{crs_raster}')
else:
    print('We must reproject vector file')
    gdf = gdf.to_crs(epsg=crs_raster.to_epsg())
    print ('vector file has been well reprojected')

CRS are the same : EPSG:PROJCRS["WGS 84 / UTM zone 31N",BASEGEOGCRS["WGS 84",ENSEMBLE["World Geodetic System 1984 ensemble",MEMBER["World Geodetic System 1984 (Transit)"],MEMBER["World Geodetic System 1984 (G730)"],MEMBER["World Geodetic System 1984 (G873)"],MEMBER["World Geodetic System 1984 (G1150)"],MEMBER["World Geodetic System 1984 (G1674)"],MEMBER["World Geodetic System 1984 (G1762)"],MEMBER["World Geodetic System 1984 (G2139)"],MEMBER["World Geodetic System 1984 (G2296)"],ELLIPSOID["WGS 84",6378137,298.257223563,LENGTHUNIT["metre",1]],ENSEMBLEACCURACY[2.0]],PRIMEM["Greenwich",0,ANGLEUNIT["degree",0.0174532925199433]],ID["EPSG",4326]],CONVERSION["UTM zone 31N",METHOD["Transverse Mercator",ID["EPSG",9807]],PARAMETER["Latitude of natural origin",0,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8801]],PARAMETER["Longitude of natural origin",3,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8802]],PARAMETER["Scale factor at natural origin",0.9996,SCALEUNIT["unity",1],ID["EPSG",880

Applying buffer

In [5]:
gdf.geometry = gdf.geometry.buffer(-30)

print(f'The Coordinates Reference System is {gdf.crs} \n')

print(f'There are {len(gdf)} polygons in the EnMAP area BEFORE applying the {-30}m buffer.')

gdf.geometry = gdf.geometry.buffer(-30)

gdf = gdf[~gdf.geometry.is_empty]    # Remove empty geometries

print(f'There are {len(gdf)} polygons AFTER applying the {30}m buffer.')

The Coordinates Reference System is EPSG:32631 

There are 20256 polygons in the EnMAP area BEFORE applying the -30m buffer.
There are 6167 polygons AFTER applying the 30m buffer.


filter gdf dataset

In [6]:
code_list = ['311', '321', '331', '341', '351', '36']

gdf_filtered = gdf.loc[gdf['CULT_COD'].isin(code_list)]
gdf_filtered['area'] = gdf_filtered.geometry.area.astype(int)

gdf_filtered = gdf_filtered.reset_index()

Compute real area of each polygon, taking into account no data values

In [8]:
'''
src = rasterio.open(EnMAP_glob_path, "r")
profile = src.profile
profile.update(dtype=rasterio.float32, 
               count = 1, 
               compress = 'lzw')
im_arr = src.read(3)

src_cloud = rasterio.open(glob_mask_path)

CLOUD_MASK = src_cloud.read(1)
im_masked = CLOUD_MASK * im_arr
'''

src = rasterio.open(EnMAP_glob_path, "r")
profile = src.profile
profile.update(dtype=rasterio.float32, 
               count = 1, 
               compress = 'lzw')

im_masked = src.read(3)

columns = ['index']
stats = ['count', 'mean', 'std', 'nodata']

# Get transform from profile (metadata)
profile = src.profile
transform = profile['transform']

src.close()

df_masked = pd.DataFrame(zonal_stats(gdf_filtered,
                                    im_masked,
                                    affine=transform,
                                    stats=['count', 'mean', 'std', 'nodata'],
                                    nodata=0,
                                    geojson_out=True,
                                    ))

df_masked = pd.json_normalize(df_masked['properties'])[columns + stats]
df_masked = df_masked.loc[df_masked['count']>0]

src.close()

In [9]:
gdf_masked = pd.merge(gdf_filtered, df_masked, on = 'index', how = 'inner')
gdf_masked['real_area'] = gdf_masked['area']*(gdf_masked['count']/(gdf_masked['count']+gdf_masked['nodata']))

Split 80/20

In [10]:
# Shuffle the GeoDataFrame rows
gdf_masked = gdf_masked.sample(frac=1, random_state=42).reset_index(drop=True)

# Function to perform the split for a specific class
def split_class(gdf, desired_area_fraction=0.8):
    # Calculate the total area for the class
    total_area_class = gdf['real_area'].sum()
    
    # Desired area for the 80% subset
    desired_area = desired_area_fraction * total_area_class
    
    # Initialize variables to track the cumulative area and the index to split
    cumulative_area = 0
    split_index = 0
    
    # Iterate over the rows to find the split point
    for i, area in enumerate(gdf['real_area']):
        cumulative_area += area
        if cumulative_area >= desired_area:
            split_index = i + 1
            break
    
    # Split the GeoDataFrame
    gdf_80 = gdf.iloc[:split_index]
    gdf_20 = gdf.iloc[split_index:]
    
    # Ensure there's at least 1 polygon in the 20% subset if the class has more than 1 polygon
    if len(gdf_20) == 0 and len(gdf) > 1:
        gdf_80 = gdf.iloc[:-1]
        gdf_20 = gdf.iloc[[-1]]  # Move the last polygon from the 80% to 20%

    return gdf_80, gdf_20

# List to store the subsets
gdf_80_list = []
gdf_20_list = []

# Loop over each class
for cult_cod in gdf_masked['CULT_COD'].unique():
    # Filter the GeoDataFrame for the current class
    gdf_class = gdf_masked[gdf_masked['CULT_COD'] == cult_cod]
    
    # Split the class-specific GeoDataFrame
    gdf_80_class, gdf_20_class = split_class(gdf_class)
    
    # Append the results to the lists
    gdf_80_list.append(gdf_80_class)
    gdf_20_list.append(gdf_20_class)

# Concatenate all the class-specific subsets to form the final GeoDataFrames
gdf_80 = pd.concat(gdf_80_list).reset_index(drop=True)
gdf_20 = pd.concat(gdf_20_list).reset_index(drop=True)

# Print the results
total_area = gdf_masked['real_area'].sum()
desired_area = 0.8 * total_area

print(f'Total area: {total_area}')
print(f'Desired 80% area: {desired_area}')
print(f'Actual 80% subset area: {gdf_80["real_area"].sum()}')
print(f'Number of polygons in 80% subset: {len(gdf_80)}')
print(f'Number of polygons in 20% subset: {len(gdf_20)}')

# Verify the split for each class
for cult_cod in gdf_masked['CULT_COD'].unique():
    class_total_area = gdf_masked[gdf_masked['CULT_COD'] == cult_cod]['real_area'].sum()
    class_80_area = gdf_80[gdf_80['CULT_COD'] == cult_cod]['real_area'].sum()
    class_20_area = gdf_20[gdf_20['CULT_COD'] == cult_cod]['real_area'].sum()
    class_80_polygons = len(gdf_80[gdf_80['CULT_COD'] == cult_cod])
    class_20_polygons = len(gdf_20[gdf_20['CULT_COD'] == cult_cod])
    print(f'Class {cult_cod}:')
    print(f'  Total area: {class_total_area}')
    print(f'  80% area: {class_80_area}')
    print(f'  20% area: {class_20_area}')
    print(f'  Number of polygons in 80% subset: {class_80_polygons}')
    print(f'  Number of polygons in 20% subset: {class_20_polygons}')


Total area: 29008395.001655277
Desired 80% area: 23206716.00132422
Actual 80% subset area: 23284272.792550378
Number of polygons in 80% subset: 1406
Number of polygons in 20% subset: 349
Class 351:
  Total area: 152814.34
  80% area: 125621.34
  20% area: 27193.0
  Number of polygons in 80% subset: 13
  Number of polygons in 20% subset: 6
Class 311:
  Total area: 24847830.256773323
  80% area: 19883345.649392564
  20% area: 4964484.607380763
  Number of polygons in 80% subset: 1171
  Number of polygons in 20% subset: 284
Class 321:
  Total area: 3712308.4048819514
  80% area: 3009743.803157814
  20% area: 702564.6017241379
  Number of polygons in 80% subset: 194
  Number of polygons in 20% subset: 54
Class 36:
  Total area: 246916.0
  80% area: 223851.0
  20% area: 23065.0
  Number of polygons in 80% subset: 24
  Number of polygons in 20% subset: 3
Class 331:
  Total area: 27123.0
  80% area: 21886.0
  20% area: 5237.0
  Number of polygons in 80% subset: 2
  Number of polygons in 20% s

In [11]:
gdf_80 = gpd.GeoDataFrame(gdf_80, crs="EPSG:32631", geometry=gdf_80['geometry'])
gdf_20 = gpd.GeoDataFrame(gdf_20, crs="EPSG:32631", geometry=gdf_20['geometry'])

In [13]:
gdf_80.to_file('images/out/80.shp')

gdf_20.to_file('images/out/20.shp')

import masked images

In [18]:
EnMAP_masked_path = 'images/EnMAP_resampled_notglob.tif'

Cropping images with plots

EnMAP

In [19]:
output_image_path = 'images/out/EnMAP_20.tif'

# Open the source raster
with rasterio.open(EnMAP_masked_path) as src:
    # Read the GeoDataFrame with the polygons
    geoms = gdf_20.geometry.values
    
    # Create a mask
    out_image, out_transform = mask(src, geoms, crop=False, nodata=-10000)
    
    # Update metadata
    out_meta = src.meta.copy()
    out_meta.update({"driver": "GTiff",
                     "height": out_image.shape[1],
                     "width": out_image.shape[2],
                     "transform": out_transform,
                     "nodata": -10000})

# Write the masked raster to a new file
with rasterio.open(output_image_path, "w", **out_meta) as dst:
    for i in range(224):
        dst.write(out_image[i], i + 1)  # i + 1 is the band index


src.close()
dst.close()


In [20]:
output_image_path = 'images/out/EnMAP_80.tif'

# Open the source raster
with rasterio.open(EnMAP_masked_path) as src:
    # Read the GeoDataFrame with the polygons
    geoms = gdf_80.geometry.values
    
    # Create a mask
    out_image, out_transform = mask(src, geoms, crop=False, nodata=-10000)
    
    # Update metadata
    out_meta = src.meta.copy()
    out_meta.update({"driver": "GTiff",
                     "height": out_image.shape[1],
                     "width": out_image.shape[2],
                     "transform": out_transform,
                     "nodata": -10000})

# Write the masked raster to a new file
with rasterio.open(output_image_path, "w", **out_meta) as dst:
    for i in range(224):
        dst.write(out_image[i], i + 1)  # i + 1 is the band index


src.close()
dst.close()

Rasterizing polygons

In [21]:
img_ref = 'images/out/EnMAP_20.tif'
rasterized_20_path = 'images/out/rasterized_20.tif'
no_data = -10000

# Open the source image to get metadata
src = rasterio.open(img_ref, "r")
out_meta = src.meta.copy()
out_meta.update({
    "count": 2,  # We want 2 bands in the output
    "nodata": no_data
})

# Create the output raster with 2 bands
dst = rasterio.open(rasterized_20_path, 'w+', **out_meta)

# Prepare arrays for the two bands
cult_cod_arr = dst.read(1, out_shape=(src.height, src.width))
index_arr = dst.read(1, out_shape=(src.height, src.width))

# This is where we create a generator of geom, value pairs to use in rasterizing
geom_col = gdf_20.geometry 
code_col = gdf_20['CULT_COD'].astype(int)
index_col = gdf_20.index

# Create shapes generators
shapes_cult_cod = ((geom, value) for geom, value in zip(geom_col, code_col))
shapes_index = ((geom, index) for geom, index in zip(geom_col, index_col))

# Rasterize CULT_COD band
cult_cod_arr = features.rasterize(shapes=shapes_cult_cod,
                                  fill=no_data,
                                  out=cult_cod_arr,
                                  transform=dst.transform)

# Rasterize index band
index_arr = features.rasterize(shapes=shapes_index,
                               fill=no_data,
                               out=index_arr,
                               transform=dst.transform)

# Write the two bands to the output file
dst.write_band(1, cult_cod_arr)
dst.write_band(2, index_arr)

print('Rasterization is done')

# Close rasterio objects
src.close()
dst.close()


Rasterization is done


In [22]:
img_ref = 'images/out/EnMAP_80.tif' ## to be change 20/80
rasterized_80_path = 'images/out/rasterized_80.tif' ## to be change 20/80
no_data = -10000

# Open the source image to get metadata
src = rasterio.open(img_ref, "r")
out_meta = src.meta.copy()
out_meta.update({
    "count": 2,  # We want 2 bands in the output
    "nodata": no_data
})

# Create the output raster with 2 bands
dst = rasterio.open(rasterized_80_path, 'w+', **out_meta) ## to be change 20/80

# Prepare arrays for the two bands
cult_cod_arr = dst.read(1, out_shape=(src.height, src.width))
index_arr = dst.read(1, out_shape=(src.height, src.width))

# This is where we create a generator of geom, value pairs to use in rasterizing
geom_col = gdf_80.geometry ## to be change 20/80
code_col = gdf_80['CULT_COD'].astype(int) ## to be change 20/80
index_col = gdf_80.index ## to be change 20/80

# Create shapes generators
shapes_cult_cod = ((geom, value) for geom, value in zip(geom_col, code_col))
shapes_index = ((geom, index) for geom, index in zip(geom_col, index_col))

# Rasterize CULT_COD band
cult_cod_arr = features.rasterize(shapes=shapes_cult_cod,
                                  fill=no_data,
                                  out=cult_cod_arr,
                                  transform=dst.transform)

# Rasterize index band
index_arr = features.rasterize(shapes=shapes_index,
                               fill=no_data,
                               out=index_arr,
                               transform=dst.transform)

# Write the two bands to the output file
dst.write_band(1, cult_cod_arr)
dst.write_band(2, index_arr)

print('Rasterization is done')

# Close rasterio objects
src.close()
dst.close()


Rasterization is done
